In [ ]:
from google .colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16

In [ ]:
print(os.listdir("/content/drive/MyDrive/Colab Notebooks/archive (1)/dataset_FVC2000_DB4_B/dataset/train_data"))

In [ ]:
SIZE = 224

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
train_images = []
train_labels = []
for directory_path in glob.glob("/content/drive/MyDrive/Colab Notebooks/archive (1)/dataset_FVC2000_DB4_B/dataset/train_data/first/*"):
    label = directory_path.split("\\")[-1]
    #print(label)
    for img_path in glob.glob(os.path.join(directory_path)):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        img = cv2.medianBlur(img,5) #filter
        #img  = sigmentation(img) #segmnetation
        cv2_imshow(img)
        train_images.append(img)
        train_labels.append(label)

In [ ]:
train_images = np.array(train_images)
print(train_images)
train_labels = np.array(train_labels)
print(train_labels)

In [ ]:
test_images = []
test_labels = []
for directory_path in glob.glob("/content/drive/MyDrive/Colab Notebooks/archive (1)/dataset_FVC2000_DB4_B/dataset/train_data/second/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path)):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))

        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)

In [ ]:
test_images = np.array(test_images)
print(test_images)
test_labels = np.array(test_labels)
print(test_labels)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
from keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
VGG_model.input

In [ ]:
for layer in VGG_model.layers:
	layer.trainable = False

In [ ]:
x = Flatten()(VGG_model.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=VGG_model.input, outputs=prediction)
model.summary()

In [ ]:
feature_extractor=VGG_model.predict(x_train)

In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
X_for_RF = features

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
RF_model.fit(X_for_RF, y_train)

In [ ]:
X_test_feature = VGG_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
prediction_RF = RF_model.predict(X_test_features)

In [ ]:
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_labels, prediction_RF)

In [ ]:
sns.heatmap(cm, annot=True)

In [ ]:
n=np.random.randint(0, x_test.shape[0])
img = x_test[n]


nt("The actual label for this image is: ", test_labels[n])